# インポート

In [1]:
import json

In [2]:
import boto3

In [3]:
from boto3.session import Session
from botocore.config import Config

# 各種情報の定義

In [4]:
endpoint = 'http://172.24.32.250:9000'

In [5]:
my_config = Config(
    signature_version = 'v4',
)

以下のポリシーは、事前に設定したバケットに対するポリシーとは異なり、今回の一時アカウント発行用のものである。

In [6]:
policy = {
    "version": "2012-10-17",
    "Statement": [
        {
            "Action": [
                "s3:GetObject",
                "s3:PutObject"
            ],
            "Effect": "Allow",
            "Resource": [
                "arn:aws:s3:::share/*"
            ],
            "Sid": "Stmt1"
        }
    ]
}

# クライアント取得、一時アカウントの発行

予め作成し、ポリシー設定しておいたアクセスキー（ユーザ名）とシークレットキーを利用する。
リージョンを指定しないとエラーになったので、仮で指定。

In [7]:
client = boto3.client("sts",
                      aws_access_key_id = 'bob',
                      aws_secret_access_key = 'bob123456',
                      region_name='us-west-2',
                      config=my_config, endpoint_url=endpoint)

In [8]:
response = client.assume_role(
    RoleArn='arn:xxx:xxx:xxx:xxxxx',
    RoleSessionName='any',
    Policy=json.dumps(policy, separators=(',', ':')),
    DurationSeconds=900,
)

なお、上記の通りRoleArn、RoleSessionNameは適当な値を入れておく。

署名済みURL発行のため、必要な値を取り出す。

In [9]:
sts = {
    'aws_access_key_id': response['Credentials']['AccessKeyId'],
    'aws_secret_access_key': response['Credentials']['SecretAccessKey'],
    'aws_session_token': response['Credentials']['SessionToken']   
}

一時アカウントの情報を用いて、セッションをインスタンス化

In [10]:
session = Session(**sts)

セッションを利用し、s3のクライアントをインスタンス化

In [11]:
s3 = session.client(
    's3',
    config=Config(signature_version='s3v4'),
    endpoint_url=endpoint
)

署名済みURLの発行

In [12]:
url = s3.generate_presigned_url(
    ClientMethod='get_object',
    Params={'Bucket': 'share', 'Key': 'test.txt'},
    ExpiresIn=60,
    HttpMethod='GET'
)

署名済みURLの保存。

In [13]:
output = 'pre-signed_url.txt'
with open(output, 'w') as f:
    f.write(url)

ローカルファイルシステムに出力されたファイル内にURLが記載されているのでブラウザなどでアクセスするとよい。

以下は、試しにこのノートブック内でアクセスする例

In [14]:
import urllib.request

In [15]:
try:
    with urllib.request.urlopen(url) as response:
        body = response.read()
        headers = response.getheaders()
        status = response.getcode()
        
        print(body)
except urllib.error.URLError as e:
    print(e.reason)

b'hoge\n'
